In [1]:
%reset -f

# jupyter_pwd = %pwd
# if jupyter_pwd == "/":
#     %cd /workspace


# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

DEBUG:__main__:hello


# Inclusive

In [2]:
train_lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343]
train_lulesh_iterations: list[int] = [8, 16, 32, 64, 128]
train_lulesh_sizes: list[int] = [16, 24, 32, 48, 64]

test_lulesh_processes: list[int] = [512]
test_lulesh_iterations: list[int] = [256]
test_lulesh_sizes: list[int] = [128]

trainDF_lulesh: pd.DataFrame = return_rawDF_lulesh(
    list_process=train_lulesh_processes,
    list_iteration=train_lulesh_iterations,
    list_size=train_lulesh_sizes,
    csvDir=csvDirPath,
)

# InclusivePerCall列の生成
trainDF_lulesh["InclusivePerCall"] = -1
trainDF_lulesh = trainDF_lulesh.reset_index()
for i, sr in trainDF_lulesh.iterrows():
    trainDF_lulesh.at[i, "InclusivePerCall"] = convertPprofTime(
        sr["Inclusive"]
    ) / float(sr["#Call"])
trainDF_lulesh = trainDF_lulesh.sort_values("#Call")
# Inclusive列の整形
_before_converted: list[float] = list(trainDF_lulesh["Inclusive"])
_after_converted: list[float] = map(convertPprofTime, _before_converted)
trainDF_lulesh["Inclusive"] = list(_after_converted)

testDF_lulesh: pd.DataFrame = return_rawDF_lulesh(
    list_process=test_lulesh_processes,
    list_iteration=test_lulesh_iterations,
    list_size=test_lulesh_sizes,
    csvDir=csvDirPath,
)

# InclusivePerCall列の生成
testDF_lulesh["InclusivePerCall"] = -1
testDF_lulesh = testDF_lulesh.reset_index()
for i, sr in testDF_lulesh.iterrows():
    testDF_lulesh.at[i, "InclusivePerCall"] = convertPprofTime(sr["Inclusive"]) / float(
        sr["#Call"]
    )
testDF_lulesh = testDF_lulesh.sort_values("#Call")

# Inclusive列の整形
_before_converted: list[float] = list(testDF_lulesh["Inclusive"])
_after_converted: list[float] = map(convertPprofTime, _before_converted)
testDF_lulesh["Inclusive"] = list(_after_converted)

functionNames: list[str] = sorted(list(set(trainDF_lulesh["Name"])))
list_series: list[pd.Series] = []
for functionName in functionNames:

    trainDF_perFunc: pd.DataFrame = trainDF_lulesh[
        trainDF_lulesh["Name"] == functionName
    ]
    testDF_perFunc: pd.DataFrame = testDF_lulesh[testDF_lulesh["Name"] == functionName]

    expVar: list[str] = ["process", "iteration", "size"]
    resVar: str

    dict_symbols = {}
    for elem in expVar:
        dict_symbols[elem] = symbols(elem, real=True)
    target_env = [
        (dict_symbols["size"], test_lulesh_sizes[0]),
        (dict_symbols["iteration"], test_lulesh_iterations[0]),
        (dict_symbols["process"], test_lulesh_processes[0]),
    ]

    # 総実行時間

    # Extra-Pへの入力ファイルの作榮
    resVar = "Inclusive"
    str_ExtraPinputData: str = gen_ExtraPinputDataFromDF(
        inputDF=trainDF_perFunc,
        expVar=expVar,
        resVar=resVar,
    )
    filePath: str = f"./extra-p_docker/share/input_lulesh_perFunc.txt"
    with open(filePath, mode="w") as f:
        f.write(str_ExtraPinputData)

    # Extra-Pの実行とその出力の取得
    res_str: str = subprocess.run(
        "extrap --text ./extra-p_docker/share/input_lulesh_perFunc.txt  | grep Model",
        stdout=subprocess.PIPE,
        text=True,
        shell=True,
    ).stdout

    # 取得したExtra-Pの出力の整形
    res_str = res_str.replace("Model: ", "")
    res_str = convert_log(res_str)

    model_sympy = sympify(res_str, locals=dict_symbols)
    predicted_all = model_sympy.subs(target_env).evalf()

    # print(predicted_all)

    # 1コール当たりの実行時間

    # Extra-Pへの入力ファイルの作榮
    resVar = "InclusivePerCall"
    str_ExtraPinputData: str = gen_ExtraPinputDataFromDF(
        inputDF=trainDF_perFunc,
        expVar=expVar,
        resVar=resVar,
    )
    filePath: str = f"./extra-p_docker/share/input_lulesh_perFunc.txt"
    with open(filePath, mode="w") as f:
        f.write(str_ExtraPinputData)

    # Extra-Pの実行とその出力の取得
    res_str: str = subprocess.run(
        "extrap --text ./extra-p_docker/share/input_lulesh_perFunc.txt  | grep Model",
        stdout=subprocess.PIPE,
        text=True,
        shell=True,
    ).stdout

    # 取得したExtra-Pの出力の整形
    res_str = res_str.replace("Model: ", "")
    res_str = convert_log(res_str)

    model_sympy = sympify(res_str, locals=dict_symbols)
    predicted_perFunc = model_sympy.subs(target_env).evalf()

    # print(predicted_perFunc)

    # 関数コール回数の予測

    # 関数コール回数予測のためのモデルを構築
    resVar = "#Call"
    result_series_list: list[pd.DataFrame] = []
    list_modelName: list[str] = [
        "modelIp",
        "modelLog",
        "modelLinAndIp",
        "modelLinAndLog",
        "modelIpAndLin",
        "modelIpAndLog",
        "modelLogAndLin",
        "modelLogAndIp",
        "modelProcessDividedByProblemSize",
        "modelProblemSizeDividedByProcess",
        "modelInfiniteProductOfProblemSizeMultipliedByProcesses",
        "modelInfiniteProductOfProblemSizeDividedByProcesses",
        "modelLinearSumOf2elementCombination",
        "modelLinearSumOfElementCombinations",
        "modelLinearSumOf2elementCombinationWithSquared",
        "modelLinearSumOf2elementCombinationWithCubed",
        "modelSquareRootOfProcess",
        "modelSquareRootTimesOtherElems",
        "modelObeyOneParameter",
        "modelLin"
        # "modelBasicTree",
    ]
    trainDF_perFunc = trainDF_perFunc.rename({"Name": "functionName"}, axis="columns")
    testDF_perFunc = testDF_perFunc.rename({"Name": "functionName"}, axis="columns")

    bestModelDict: dict = return_bestModelObject(
        inputDF=trainDF_perFunc,
        list_expVar=expVar,
        list_resVar=[resVar],
        list_modelName=list_modelName,
    )

    bestModel = bestModelDict["object"]

    predicted = float(np.array(bestModel.predict(inputDF=testDF_perFunc[expVar])))
    _call: float = float(testDF_perFunc.iloc[0][resVar])

    # print(_call)

    # 実際の値の取得と結果の整理

    real_time: float = testDF_perFunc.reset_index().loc[0]["Inclusive"]
    real_call: float = testDF_perFunc.reset_index().loc[0]["#Call"]

    _series: pd.Series = pd.Series(
        {
            "functionName": functionName,
            "real_time": real_time,
            "predicted_all": predicted_all,
            "predicted_from_perCall": predicted_perFunc * predicted,
            "real_call": real_call,
            "predicted_call": predicted,
        }
    )

    # print(_series)

    list_series.append(_series)

Loading file: 100%|██████████| [00:00<00:00, Validating experiment]
Generating models: 100%|██████████| [00:00<00:00]
Loading file: 100%|██████████| [00:00<00:00, Validating experiment]
Generating models: 100%|██████████| [00:00<00:00]
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
Loading file: 100%|██████████| [00:00<00:00, Validating experiment]
Generating models: 100%|██████████| [00:00<00:00]
Loading file: 100%|██████████| [00:00<00:00, Validating experiment]
Generating models: 100%|██████████| [00:00<00:00]
Loading file: 100%|██████████| [00:00<00:00, Validating experiment]
Generating models: 100%|██████

In [3]:
result_InclusiveLulesh = pd.concat(list_series, axis=1).T
result_InclusiveLulesh = add_relativeErrorRateCol(inputDF=result_InclusiveLulesh, real_colName = "real_time", predicted_colName = "predicted_all", targetColName = "relativeError_real_vs_all")
result_InclusiveLulesh = add_relativeErrorRateCol(inputDF=result_InclusiveLulesh, real_colName = "real_time", predicted_colName = "predicted_from_perCall", targetColName = "relativeError_real_vs_perCall")
result_InclusiveLulesh

,functionName,real_time,predicted_all,predicted_from_perCall,real_call,predicted_call,relativeError_real_vs_all,relativeError_real_vs_perCall
0,.TAU_application,1149.925,1288.20034616662,1288.20034616662,1.0,1.0,12.0247273662734,12.0247273662734
1,MPI_Allreduce(),110.861,155.571448846150,184.113257775612,255.0,255.0,40.3301872129514,66.0757685530640
2,MPI_Barrier(),0.039,0.00613370466666667,0.00613370466666667,1.0,1.0,84.2725521367521,84.2725521367521
3,MPI_Comm_rank(),0.002,0.00340661419639977,0.00249937972674534,2309.0,2309.0,70.3307098199886,24.9689863372672
4,MPI_Comm_size(),0.0,4.66073333333333e-7,4.66073333333333e-7,1.0,1.0,4.88299319727895,4.88299319727895
5,MPI_Finalize(),0.653,-0.877687749034840,-0.877687749034840,1.0,1.0,234.408537371338,234.408537371338
6,MPI_Init(),0.593,0.368159207096530,0.368159207096530,1.0,1.0,37.9158166784941,37.9158166784941
7,MPI_Irecv(),0.018,0.0237588378914084,0.0173408882000237,8965.8,9907.354054,31.9935438411581,3.66173222209041
8,MPI_Isend(),0.248,1.54147758517037,3.31738815983449,8965.8,9907.354054,521.563542407407,1237.65651606229
9,MPI_Reduce(),0.069,0.115818278291975,0.115818278291975,1.0,1.0,67.8525772347463,67.8525772347463


In [4]:
highErrorInclusive :pd.DataFrame = result_InclusiveLulesh[result_InclusiveLulesh["relativeError_real_vs_all"] < result_InclusiveLulesh["relativeError_real_vs_perCall"]]
print(highErrorInclusive.to_csv())

,functionName,real_time,predicted_all,predicted_from_perCall,real_call,predicted_call,relativeError_real_vs_all,relativeError_real_vs_perCall
1,MPI_Allreduce(),110.86099999999999,155.571448846150,184.113257775612,255.0,255.0,40.3301872129514,66.0757685530640
8,MPI_Isend(),0.248,1.54147758517037,3.31738815983449,8965.8,9907.354054181274,521.563542407407,1237.65651606229
10,MPI_Wait(),12.869,58.1663402488279,90.9780806259428,8965.8,9907.354053711444,351.988035191762,606.955323847563
15,void_CalcKinematicsForElems(Domain,192.675,240.505983855522,142.224102799915,256.0,256.0,24.8246964346814,26.1844542364528
17,void_CommRecv(Domain,0.023,0.0185219547238595,0.0288996380981962,769.0,768.9999999999998,19.4697620701763,25.6506004269401
18,void_CommSBN(Domain,7.184,1.06010835189704,43.9930687012667,257.0,257.0,85.2434806250413,512.375677912955
20,void_CommSyncPosVel(Domain,7.888,17.2230836208581,40.1641976811198,256.0,256.0,118.345380588972,409.181005085190



In [5]:
def changeDigits(inputNum: int | float) -> str:
    return str(int(inputNum * 1000) / 1000)


print(
    result_InclusiveLulesh.style.format(
        formatter={
            "real_time": changeDigits,
            "predicted_all": changeDigits,
            "predicted_from_perCall": changeDigits,
            "real_call": changeDigits,
            "predicted_call": changeDigits,
        },
        escape="latex"
    ).to_latex(
        column_format="lp{1cm}p{1cm}SSSS",
        position="b",
        label=f"2022年9月14日_Inc",
        caption=f"LULESHによる予測結果",
        siunitx=True,
    )
)

\begin{table}[b]
\caption{LULESHによる予測結果}
\label{2022年9月14日_Inc}
\begin{tabular}{lp{1cm}p{1cm}SSSS}
{} & {functionName} & {real_time} & {predicted_all} & {predicted_from_perCall} & {real_call} & {predicted_call} & {relativeError_real_vs_all} & {relativeError_real_vs_perCall} \\
0 & .TAU\_application & 1149.925 & 1288.2 & 1288.2 & 1.0 & 1.0 & 12.0247273662734 & 12.0247273662734 \\
1 & MPI\_Allreduce() & 110.86 & 155.571 & 184.113 & 255.0 & 255.0 & 40.3301872129514 & 66.0757685530640 \\
2 & MPI\_Barrier() & 0.039 & 0.006 & 0.006 & 1.0 & 1.0 & 84.2725521367521 & 84.2725521367521 \\
3 & MPI\_Comm\_rank() & 0.002 & 0.003 & 0.002 & 2309.0 & 2308.999 & 70.3307098199886 & 24.9689863372672 \\
4 & MPI\_Comm\_size() & 0.0 & 0.0 & 0.0 & 1.0 & 1.0 & 4.88299319727895 & 4.88299319727895 \\
5 & MPI\_Finalize() & 0.653 & -0.877 & -0.877 & 1.0 & 1.0 & 234.408537371338 & 234.408537371338 \\
6 & MPI\_Init() & 0.593 & 0.368 & 0.368 & 1.0 & 1.0 & 37.9158166784941 & 37.9158166784941 \\
7 & MPI\_Irecv() & 0.01

In [6]:
formatted_result_InclusiveLulesh :pd.DataFrame = result_InclusiveLulesh.rename(columns={"functionName":"関数名", "real_time":"実時間", "predicted_all":"Extra-Pのみを利用して予測した時間", "predicted_from_perCall":"Extra-Pと提案手法を組み合わせた予測した時間", "real_call":"実コール回数", "predicted_call":"提案手法にて予測した関数コール回数"})
print(formatted_result_InclusiveLulesh.to_csv())

,関数名,実時間,Extra-Pのみを利用して予測した時間,Extra-Pと提案手法を組み合わせた予測した時間,実コール回数,提案手法にて予測した関数コール回数,relativeError_real_vs_all,relativeError_real_vs_perCall
0,.TAU_application,1149.925,1288.20034616662,1288.20034616662,1.0,1.0,12.0247273662734,12.0247273662734
1,MPI_Allreduce(),110.86099999999999,155.571448846150,184.113257775612,255.0,255.0,40.3301872129514,66.0757685530640
2,MPI_Barrier(),0.039,0.00613370466666667,0.00613370466666667,1.0,1.0,84.2725521367521,84.2725521367521
3,MPI_Comm_rank(),0.002,0.00340661419639977,0.00249937972674534,2309.0,2308.9999999999995,70.3307098199886,24.9689863372672
4,MPI_Comm_size(),4.9e-07,4.66073333333333e-7,4.66073333333333e-7,1.0,1.0,4.88299319727895,4.88299319727895
5,MPI_Finalize(),0.653,-0.877687749034840,-0.877687749034840,1.0,1.0,234.408537371338,234.408537371338
6,MPI_Init(),0.593,0.368159207096530,0.368159207096530,1.0,1.0,37.9158166784941,37.9158166784941
7,MPI_Irecv(),0.018,0.0237588378914084,0.0173408882000237,8965.8,9907.354053779622,31.9935438411581,3.6617

In [7]:
testDF_perFunc.reset_index()

,level_0,index,%Time,Exclusive,Inclusive,#Call,#Subrs,functionName,process,iteration,size,InclusivePerCall
0,31,31,0.0,0.000256,2.560000e-07,0.001953,0.0,void_VerifyAndWriteFinalOutput(Real_t_Domain,512,256,128,0.000131


In [8]:
%reset -f

# jupyter_pwd = %pwd
# if jupyter_pwd == "/":
#     %cd /workspace


# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

DEBUG:__main__:hello


# Exclusive

In [9]:
train_lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343]
train_lulesh_iterations: list[int] = [8, 16, 32, 64, 128]
train_lulesh_sizes: list[int] = [16, 24, 32, 48, 64]

test_lulesh_processes: list[int] = [512]
test_lulesh_iterations: list[int] = [256]
test_lulesh_sizes: list[int] = [128]

trainDF_lulesh: pd.DataFrame = return_rawDF_lulesh(
    list_process=train_lulesh_processes,
    list_iteration=train_lulesh_iterations,
    list_size=train_lulesh_sizes,
    csvDir=csvDirPath,
)

# ExclusivePerCall列の生成
trainDF_lulesh["ExclusivePerCall"] = -1
trainDF_lulesh = trainDF_lulesh.reset_index()
for i, sr in trainDF_lulesh.iterrows():
    trainDF_lulesh.at[i, "ExclusivePerCall"] = convertPprofTime(
        sr["Exclusive"]
    ) / float(sr["#Call"])
trainDF_lulesh = trainDF_lulesh.sort_values("#Call")
# Exclusive列の整形
_before_converted: list[float] = list(trainDF_lulesh["Exclusive"])
_after_converted: list[float] = map(convertPprofTime, _before_converted)
trainDF_lulesh["Exclusive"] = list(_after_converted)

testDF_lulesh: pd.DataFrame = return_rawDF_lulesh(
    list_process=test_lulesh_processes,
    list_iteration=test_lulesh_iterations,
    list_size=test_lulesh_sizes,
    csvDir=csvDirPath,
)

# ExclusivePerCall列の生成
testDF_lulesh["ExclusivePerCall"] = -1
testDF_lulesh = testDF_lulesh.reset_index()
for i, sr in testDF_lulesh.iterrows():
    testDF_lulesh.at[i, "ExclusivePerCall"] = convertPprofTime(sr["Exclusive"]) / float(
        sr["#Call"]
    )
testDF_lulesh = testDF_lulesh.sort_values("#Call")

# Exclusive列の整形
_before_converted: list[float] = list(testDF_lulesh["Exclusive"])
_after_converted: list[float] = map(convertPprofTime, _before_converted)
testDF_lulesh["Exclusive"] = list(_after_converted)

functionNames: list[str] = sorted(list(set(trainDF_lulesh["Name"])))
list_series: list[pd.Series] = []
for functionName in functionNames:

    trainDF_perFunc: pd.DataFrame = trainDF_lulesh[
        trainDF_lulesh["Name"] == functionName
    ]
    testDF_perFunc: pd.DataFrame = testDF_lulesh[testDF_lulesh["Name"] == functionName]

    expVar: list[str] = ["process", "iteration", "size"]
    resVar: str

    dict_symbols = {}
    for elem in expVar:
        dict_symbols[elem] = symbols(elem, real=True)
    target_env = [
        (dict_symbols["size"], test_lulesh_sizes[0]),
        (dict_symbols["iteration"], test_lulesh_iterations[0]),
        (dict_symbols["process"], test_lulesh_processes[0]),
    ]

    # 総実行時間

    # Extra-Pへの入力ファイルの作榮
    resVar = "Exclusive"
    str_ExtraPinputData: str = gen_ExtraPinputDataFromDF(
        inputDF=trainDF_perFunc,
        expVar=expVar,
        resVar=resVar,
    )
    filePath: str = f"./extra-p_docker/share/input_lulesh_perFunc.txt"
    with open(filePath, mode="w") as f:
        f.write(str_ExtraPinputData)

    # Extra-Pの実行とその出力の取得
    res_str: str = subprocess.run(
        "extrap --text ./extra-p_docker/share/input_lulesh_perFunc.txt  | grep Model",
        stdout=subprocess.PIPE,
        text=True,
        shell=True,
    ).stdout

    # 取得したExtra-Pの出力の整形
    res_str = res_str.replace("Model: ", "")
    res_str = convert_log(res_str)

    model_sympy = sympify(res_str, locals=dict_symbols)
    predicted_all = model_sympy.subs(target_env).evalf()

    # print(predicted_all)

    # 1コール当たりの実行時間

    # Extra-Pへの入力ファイルの作榮
    resVar = "ExclusivePerCall"
    str_ExtraPinputData: str = gen_ExtraPinputDataFromDF(
        inputDF=trainDF_perFunc,
        expVar=expVar,
        resVar=resVar,
    )
    filePath: str = f"./extra-p_docker/share/input_lulesh_perFunc.txt"
    with open(filePath, mode="w") as f:
        f.write(str_ExtraPinputData)

    # Extra-Pの実行とその出力の取得
    res_str: str = subprocess.run(
        "extrap --text ./extra-p_docker/share/input_lulesh_perFunc.txt  | grep Model",
        stdout=subprocess.PIPE,
        text=True,
        shell=True,
    ).stdout

    # 取得したExtra-Pの出力の整形
    res_str = res_str.replace("Model: ", "")
    res_str = convert_log(res_str)

    model_sympy = sympify(res_str, locals=dict_symbols)
    predicted_perFunc = model_sympy.subs(target_env).evalf()

    # print(predicted_perFunc)

    # 関数コール回数の予測

    # 関数コール回数予測のためのモデルを構築
    resVar = "#Call"
    result_series_list: list[pd.DataFrame] = []
    list_modelName: list[str] = [
        "modelIp",
        "modelLog",
        "modelLinAndIp",
        "modelLinAndLog",
        "modelIpAndLin",
        "modelIpAndLog",
        "modelLogAndLin",
        "modelLogAndIp",
        "modelProcessDividedByProblemSize",
        "modelProblemSizeDividedByProcess",
        "modelInfiniteProductOfProblemSizeMultipliedByProcesses",
        "modelInfiniteProductOfProblemSizeDividedByProcesses",
        "modelLinearSumOf2elementCombination",
        "modelLinearSumOfElementCombinations",
        "modelLinearSumOf2elementCombinationWithSquared",
        "modelLinearSumOf2elementCombinationWithCubed",
        "modelSquareRootOfProcess",
        "modelSquareRootTimesOtherElems",
        "modelObeyOneParameter",
        "modelLin"
        # "modelBasicTree",
    ]
    trainDF_perFunc = trainDF_perFunc.rename({"Name": "functionName"}, axis="columns")
    testDF_perFunc = testDF_perFunc.rename({"Name": "functionName"}, axis="columns")

    bestModelDict: dict = return_bestModelObject(
        inputDF=trainDF_perFunc,
        list_expVar=expVar,
        list_resVar=[resVar],
        list_modelName=list_modelName,
    )

    bestModel = bestModelDict["object"]

    predicted = float(np.array(bestModel.predict(inputDF=testDF_perFunc[expVar])))
    _call: float = float(testDF_perFunc.iloc[0][resVar])

    # print(_call)

    # 実際の値の取得と結果の整理

    real_time: float = testDF_perFunc.reset_index().loc[0]["Exclusive"]
    real_call: float = testDF_perFunc.reset_index().loc[0]["#Call"]

    _series: pd.Series = pd.Series(
        {
            "functionName": functionName,
            "real_time": real_time,
            "predicted_all": predicted_all,
            "predicted_from_perCall": predicted_perFunc * predicted,
            "real_call": real_call,
            "predicted_call": predicted,
        }
    )

    # print(_series)

    list_series.append(_series)

Loading file: 100%|██████████| [00:00<00:00, Validating experiment]
Generating models: 100%|██████████| [00:00<00:00]
Loading file: 100%|██████████| [00:00<00:00, Validating experiment]
Generating models: 100%|██████████| [00:00<00:00]
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
Loading file: 100%|██████████| [00:00<00:00, Validating experiment]
Generating models: 100%|██████████| [00:00<00:00]
Loading file: 100%|██████████| [00:00<00:00, Validating experiment]
Generating models: 100%|██████████| [00:00<00:00]
Loading file: 100%|██████████| [00:00<00:00, Validating experiment]
Generating models: 100%|██████

In [10]:
resultExclusiveLulesh = pd.concat(list_series, axis=1).T
resultExclusiveLulesh = add_relativeErrorRateCol(inputDF=resultExclusiveLulesh, real_colName = "real_time", predicted_colName = "predicted_all", targetColName = "relativeError_real_vs_all")
resultExclusiveLulesh = add_relativeErrorRateCol(inputDF=resultExclusiveLulesh, real_colName = "real_time", predicted_colName = "predicted_from_perCall", targetColName = "relativeError_real_vs_perCall")
resultExclusiveLulesh

,functionName,real_time,predicted_all,predicted_from_perCall,real_call,predicted_call,relativeError_real_vs_all,relativeError_real_vs_perCall
0,.TAU_application,0.000003,3.60680000000000e-6,3.60680000000000e-6,1.0,1.0,3.64367816091956,3.64367816091956
1,MPI_Allreduce(),110.861,155.571448846150,184.113257775612,255.0,255.0,40.3301872129514,66.0757685530640
2,MPI_Barrier(),0.039,0.00613370466666667,0.00613370466666667,1.0,1.0,84.2725521367521,84.2725521367521
3,MPI_Comm_rank(),0.002,0.00340661419639977,0.00249937972674534,2309.0,2309.0,70.3307098199886,24.9689863372672
4,MPI_Comm_size(),0.0,4.66073333333333e-7,4.66073333333333e-7,1.0,1.0,4.88299319727895,4.88299319727895
5,MPI_Finalize(),0.653,-0.877687749034840,-0.877687749034840,1.0,1.0,234.408537371338,234.408537371338
6,MPI_Init(),0.593,0.368159207096530,0.368159207096530,1.0,1.0,37.9158166784941,37.9158166784941
7,MPI_Irecv(),0.018,0.0237588378914084,0.0173408882000237,8965.8,9907.354054,31.9935438411581,3.66173222209041
8,MPI_Isend(),0.248,1.54147758517037,3.31738815983449,8965.8,9907.354054,521.563542407407,1237.65651606229
9,MPI_Reduce(),0.069,0.115818278291975,0.115818278291975,1.0,1.0,67.8525772347463,67.8525772347463


In [11]:
highErrorExclusive :pd.DataFrame = resultExclusiveLulesh[resultExclusiveLulesh["relativeError_real_vs_all"] < resultExclusiveLulesh["relativeError_real_vs_perCall"]]
print(highErrorExclusive.to_csv())

,functionName,real_time,predicted_all,predicted_from_perCall,real_call,predicted_call,relativeError_real_vs_all,relativeError_real_vs_perCall
1,MPI_Allreduce(),110.86099999999999,155.571448846150,184.113257775612,255.0,255.0,40.3301872129514,66.0757685530640
8,MPI_Isend(),0.248,1.54147758517037,3.31738815983449,8965.8,9907.354054181274,521.563542407407,1237.65651606229
10,MPI_Wait(),12.869,58.1663402488279,90.9780806259428,8965.8,9907.354053711444,351.988035191762,606.955323847563
15,void_CalcKinematicsForElems(Domain,113.429,142.158748848381,146.641590975902,256.0,256.0,25.3283982476980,29.2805111355142
16,void_CommMonoQ(Domain,0.345,0.533358327712401,0.597490593507110,256.0,256.0,54.5966167282323,73.1856792774231
17,void_CommRecv(Domain,0.004,0.00222866670604391,0.000221985050882297,769.0,768.9999999999998,44.2833323489023,94.4503737279426
18,void_CommSBN(Domain,0.66,0.589804844319553,0.542637782999804,257.0,257.0,10.6356296485525,17.7821540909388
20,void_CommSyncPosVel(Domain,2.773,

In [13]:
formatted_resultExclusiveLulesh :pd.DataFrame = resultExclusiveLulesh.rename(columns={"functionName":"関数名", "real_time":"実時間", "predicted_all":"Extra-Pのみを利用して予測した時間", "predicted_from_perCall":"Extra-Pと提案手法を組み合わせた予測した時間", "real_call":"実コール回数", "predicted_call":"提案手法にて予測した関数コール回数"})
print(formatted_resultExclusiveLulesh.to_csv())

,関数名,実時間,Extra-Pのみを利用して予測した時間,Extra-Pと提案手法を組み合わせた予測した時間,実コール回数,提案手法にて予測した関数コール回数,relativeError_real_vs_all,relativeError_real_vs_perCall
0,.TAU_application,3.48e-06,3.60680000000000e-6,3.60680000000000e-6,1.0,1.0,3.64367816091956,3.64367816091956
1,MPI_Allreduce(),110.86099999999999,155.571448846150,184.113257775612,255.0,255.0,40.3301872129514,66.0757685530640
2,MPI_Barrier(),0.039,0.00613370466666667,0.00613370466666667,1.0,1.0,84.2725521367521,84.2725521367521
3,MPI_Comm_rank(),0.002,0.00340661419639977,0.00249937972674534,2309.0,2308.9999999999995,70.3307098199886,24.9689863372672
4,MPI_Comm_size(),4.9e-07,4.66073333333333e-7,4.66073333333333e-7,1.0,1.0,4.88299319727895,4.88299319727895
5,MPI_Finalize(),0.653,-0.877687749034840,-0.877687749034840,1.0,1.0,234.408537371338,234.408537371338
6,MPI_Init(),0.593,0.368159207096530,0.368159207096530,1.0,1.0,37.9158166784941,37.9158166784941
7,MPI_Irecv(),0.018,0.0237588378914084,0.0173408882000237,8965.8,9907.354053779622,31.9935438411581,